<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Model-specification" data-toc-modified-id="Model-specification-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model specification</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="#Training-loop" data-toc-modified-id="Training-loop-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Training loop</a></span></li><li><span><a href="#Diagnostics" data-toc-modified-id="Diagnostics-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Diagnostics</a></span></li><li><span><a href="#Inspecting-the-summary-space" data-toc-modified-id="Inspecting-the-summary-space-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Inspecting the summary space</a></span></li></ul></li><li><span><a href="#Observed-Data:-Misspecification-Detection" data-toc-modified-id="Observed-Data:-Misspecification-Detection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Observed Data: Misspecification Detection</a></span><ul class="toc-item"><li><span><a href="#Visualization-in-data-space" data-toc-modified-id="Visualization-in-data-space-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Visualization in data space</a></span></li><li><span><a href="#Detecting-misspecification-in-summary-space" data-toc-modified-id="Detecting-misspecification-in-summary-space-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Detecting misspecification in summary space</a></span></li></ul></li><li><span><a href="#Sensitivity-to-Misspecification" data-toc-modified-id="Sensitivity-to-Misspecification-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Sensitivity to Misspecification</a></span><ul class="toc-item"><li><span><a href="#Computing-the-sensitivity" data-toc-modified-id="Computing-the-sensitivity-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Computing the sensitivity</a></span></li><li><span><a href="#Plotting-the-results" data-toc-modified-id="Plotting-the-results-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Plotting the results</a></span></li></ul></li></ul></div>

In [ ]:
import bayesflow as bf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

# Introduction

Under certain regularity conditions, the theory on simulation-based inference assures that the neural posterior estimator $q_{\phi}$ samples from the exact posterior $p(\theta\,|\,x)$ after convergence.

However, the neural posterior approximator is optimized with respect to the prior of the generative model which we specify for the training process.
When the generative model at test time (the "true data generating process") deviates from the one used during training, the guarantees for the approximate neural posterior no longer hold and the approximate posterior samples can be wrong in essentially arbitrary ways.

The precise definition of model misspecification in amortized inference, along with extensive implications and experiments, is available in the following pre-print: https://arxiv.org/abs/2112.08866

![](../images/model_misspecification_amortized_sbi.png?raw=true)

# Model specification

The general Bayesian forward model can be formulated as a two-step process:

$$
\theta \sim p(\theta) \qquad x\sim p(x|\theta)
$$


We specify a fairly simple generative model where the means of a 2-dimensional Gaussian shall be estimated: $\theta=\mu=(\mu_1, \mu_2)$. Consequently, the likelihood $p(x|\theta)$ is a Gaussian $\mathcal{N}(\mu, \Sigma)$ with location $\mu$ and covariance matrix $\Sigma$. The prior distribution $p(\theta)$ over the inference targets is again a Gaussian $\mathcal{N}(\mu_0, \Sigma_0)$ with location $\mu_0$ and covariance $\Sigma_0$.

Consequently, the forward model is

$$
\mu\sim\mathcal{N}(\mu_0, \Sigma_0) \qquad x\sim\mathcal{N}(\mu, \Sigma)
$$

with fixed parameters $\mu_0, \Sigma_0, \Sigma$, while we desire posterior inference over the parameter $\mu$.

We choose $\mu_0=0, \Sigma_0=\mathbb{I}, \Sigma=\mathbb{I}$ as fixed parameters of the generative model for training the neural posterior approximator. Each simulated data set contains $N=100$ observations:

In [ ]:
def prior(D=2, mu=0., sigma=1.0):
    """Gaussian prior random number generator."""
    return np.random.default_rng().normal(loc=mu, scale=sigma, size=D)

def simulator(theta, n_obs=100, scale=1.0):
    """Gaussian likelihood random number generator"""
    return np.random.default_rng().normal(loc=theta, scale=scale, size=(n_obs, theta.shape[0]))

generative_model = bf.simulation.GenerativeModel(prior=prior, 
                                                 simulator=simulator, 
                                                 name="Generative Model: Training",
                                                 simulator_is_batched=False)

# Training

We choose a `DeepSet` (1) to learn 2 summary statistics from the data, which are then passed to the posterior network and jointly optimized.
The Inference network is a standard `InvertibleNetwork` with four coupling layers and the `AmortizedPosterior` combines the inference and summary networks. Since we desire model misspecification detection via a structured summary space (2), we select `summary_loss_fun="MMD"` and the amortizer combines its losses correctly.
Finally, the `trainer` wraps the generative model and the amortizer into a consistent object for training and subsequent sampling.

(1) Zaheer et al (2017): https://arxiv.org/abs/1703.06114

(2) Schmitt et al (2021): https://arxiv.org/abs/2112.08866

In [ ]:
summary_net = bf.networks.DeepSet(summary_dim=2)
inference_net = bf.networks.InvertibleNetwork(num_params=2, num_coupling_layers=2)
amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net, summary_loss_fun="MMD")
trainer = bf.trainers.Trainer(generative_model=generative_model, amortizer=amortizer)

## Training loop

Because the inference problem is simple, we just train for 15 epochs with 500 iterations per epoch and a batch size of 32.

In [ ]:
losses = trainer.train_online(epochs=15, iterations_per_epoch=500, batch_size=32)

## Diagnostics

In [ ]:
_ = bf.diagnostics.plot_losses(losses)

In [ ]:
_ = trainer.diagnose_sbc_histograms()

In [ ]:
new_sims = trainer.configurator(trainer.generative_model(200))
posterior_draws = amortizer.sample(new_sims, n_samples=500)

_ = bf.diagnostics.plot_recovery(posterior_draws, new_sims['parameters'])

## Inspecting the summary space

In fact, the summary space has essentially converged to a unit Gaussian for samples from the generative model which we used to train the networks.

In [ ]:
simulations = trainer.configurator(trainer.generative_model(10000))
summary_statistics = trainer.amortizer.summary_net(simulations['summary_conditions'])
theta = simulations['parameters']

_ = bf.diagnostics.plot_latent_space_2d(summary_statistics)

# Observed Data: Misspecification Detection

After assessing the converged neural posterior approximator's performance for the generative model from training, we will now perform inference on data from a different data generating process. In a real-life analysis, this would be the observed data $x_{\text{obs}}$ from an experiment or study.

For this illustration, we choose the prior scale $\tau_0$ as the source of misspecification. That means that we observe 1000 data sets $\{x_{\text{obs}}^{(k)}\}_{k=1}^{1000}$ from a generative model with prior scale $\tau_0=4$. Consequently, the prior covariance is $4\cdot\mathbb{I}=\begin{pmatrix}4&0\\0&4\end{pmatrix}$. The remaining fixed parameters $\mu_0$ and $\Sigma$ are unaltered.

In [ ]:
def prior_obs(D=2, mu=0.0, sigma=4.0):
    """Gaussian prior random number generator."""
    return np.random.default_rng().normal(loc=mu, scale=sigma, size=D)

def simulator_obs(theta, n_obs=100, scale=1.0):
    """Gaussian likelihood random number generator"""
    return np.random.default_rng().normal(loc=theta, scale=scale, size=(n_obs, theta.shape[0]))

generative_model_obs = bf.simulation.GenerativeModel(prior=prior_obs, 
                                                     simulator=simulator_obs, 
                                                     name="Generative Model: Observed",
                                                     simulator_is_batched=False)

In [ ]:
# 1000 simulated data sets from the well-specified model from training (for reference)
simulations = trainer.configurator(trainer.generative_model(1000))
x = simulations['summary_conditions']

# 1000 "observed" data sets with different prior covariance
simulations_obs = trainer.configurator(generative_model_obs(1000))
x_obs = simulations_obs['summary_conditions']

## Visualization in data space

Let's visualize some of the data $x_{\text{obs}}$ from that generative model. This plot lives on the data domain $\mathbb{R}^2$ and depicts the data $x_{\text{obs}}$. Each color is one data set $k=1,\ldots,1000$, and all points of one color form the respective data set $x_{\text{obs}}^{(k)}$.

In [ ]:
n_data_sets_visualization = 10
colors = cm.viridis(np.linspace(0, 1, n_data_sets_visualization))
indices = list(range(n_data_sets_visualization))

fig, (ax1, ax2) = plt.subplots(1,2)
for idx, c in zip(indices, colors):
    ax1.scatter(x[idx, :, 0], x[idx, :, 1], color=c, alpha=0.7)
    ax2.scatter(x_obs[idx, :, 0], x_obs[idx, :, 1], color=c, alpha=0.7)

for ax in (ax1, ax2):
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.set_aspect('equal')
    sns.despine()
    
ax1.set_title("data from well-specified model")
ax2.set_title("observed data")

## Detecting misspecification in summary space

As proposed in the paper (2), we will detect the deviating observed data as deviations in the structured summary space. Therefore, we compute the learned summary statistics of the well-specified data $h_{\psi}(x)$ and for the observed data $h_{\psi}(x_{\text{obs}})$ by a simple pass through the trainer's summary network $h_{\psi}$.


(2) Schmitt et al (2021): https://arxiv.org/abs/2112.08866

In [ ]:
summary_statistics = trainer.amortizer.summary_net(x)
summary_statistics_obs = trainer.amortizer.summary_net(x_obs)

In [ ]:
colors = cm.viridis(np.linspace(0.1, 0.9, 2))

plt.scatter(summary_statistics_obs[:, 0], summary_statistics_obs[:, 1], color=colors[0], label=r"observed: $h_{\psi}(x_{obs})$")
plt.scatter(summary_statistics[:, 0], summary_statistics[:, 1], color=colors[1], label="well-specified: $h_{\psi}(x)$")

plt.legend()

plt.gca().set_aspect('equal')
sns.despine()

This visual discrepancy can be quantified in many ways. In this case, we choose the *Maximum Mean Discrepancy*, more specifically its biased estimator (3), as implemented in `bayesflow.computational_utilities.maximum_mean_discrepancy`.
The larger the MMD, the more do the samples deviate.

(3) Gretton (2012): https://www.jmlr.org/papers/volume13/gretton12a/gretton12a.pdf

In [ ]:
from bayesflow import computational_utilities

mmd = computational_utilities.maximum_mean_discrepancy(s, s_obs)

print(f"Estimated MMD in summary space: {mmd:.3f}")

# Sensitivity to Misspecification

The submodule `bayesflow.sensitivity` contains functions to analyze the sensitivity of a converged `Trainer` (i.e., the neural posterior estimator) to model misspecification.

We start by redefining the generative model with the possibility to increase the model's misspecification through two settings `p1` and `p2`. Therefore, we define a function `generative_model_misspecified(p1, p2)`. The function takes the settings `p1` and `p2` as input and returns a (potentially misspecified) generative model.

In our Gaussian example, we let `p1` control the prior location ($\mu_0=\mathtt{p1}$) while `p2` controls the scale of a diagonal covariance matrix $\Sigma_0$ such that $\Sigma_0=\mathtt{p2}\cdot\mathbb{I}$. In this example, both settings cause prior misspecification. Inducing other types of misspecification (e.g., simulator or noise) follows the same principle.

The consequence: If `p1=0` and `p2=1`, the `generative_model_misspecified` function yields the original well-specified model from training. For all other values for `p1` and `p2`, the resulting generative model differs.

**Implementation details:** 

- The `partial` application pattern lets us pre-load the `prior` with custom arguments and pass this pre-loaded function into the generative model. We use this technique to use `p1` and `p2` as parameters in the prior callable.
- We skip the generative model's consistency checks and setup outputs via `skip_test=True`.

In [ ]:
from functools import partial

def generative_model_misspecified(p1, p2):
    prior_ = partial(prior, D=2, mu=p1, sigma=p2)
    simulator_ = partial(simulator, scale=1.0)
    generative_model_ = bf.simulation.GenerativeModel(prior_, simulator_, simulator_is_batched=False, skip_test=True)
    return generative_model_

In the next step, we provide meta-information for the sensitivity analysis:

- Names of the settings `p1` and `p2`: proper axis labels
- Range of the settings `p1` and `p2`: defining the experiment's grid
- well-specified value for the settings `p1` and `p2` (i.e., `p1=0` and `p2=1` in our example): dashed lines for the baseline configuration in the plots

In [ ]:
p1_config = {
    "name": r"$\mu_0$ (prior location)",
    "values": np.linspace(-0.1, 3.1, num=20),
    "well_specified_value": 0.0,
}
p2_config = {
    "name": r"$\tau_0$ (prior scale)",
    "values": np.linspace(0.1, 10.1, num=20),
    "well_specified_value": 1.0,
}

## Computing the sensitivity

As described above, the `bf.sensitivity.misspecification_experiment` function requires the converged `Trainer`, the factory for misspecified models, and meta-information on the settings. In addition, the number of posterior samples per simulated data set as well as the total number of simulated data sets per setting configuration can be specified.

In [ ]:
posterior_error, summary_mmd = bf.sensitivity.misspecification_experiment(
    trainer=trainer,
    generator=generative_model_misspecified,
    first_config_dict=p1_config,
    second_config_dict=p2_config,
    n_posterior_samples=500,
    n_sim=100,
)

## Plotting the results

Model misspecification with respect to both prior location $\mu_0$ and scale $\tau_0$ worsen the average posterior recovery in terms of aggregated RMSE. However, the converged posterior approximator seems to be relatively robust against moderate misspecifications in these parameters.

In [ ]:
_ = bf.sensitivity.plot_model_misspecification_sensitivity(posterior_error, p1_config, p2_config,
                                                          plot_config={'vmin': None})

The MMD plot clearly shows that the summary space MMD is lowest when the model is well-specified (coordinates `(0, 1)`). When either the prior location $\mu_0$ or the prior scale $\tau_0$ changes, the summary MMD increases and we're alerted of the model misspecification.

In [ ]:
_ = bf.sensitivity.plot_model_misspecification_sensitivity(summary_mmd, p1_config, p2_config,
                                                          plot_config={'vmin': None})